In [1]:
#load "Paket.fsx"

In [2]:
let wc = new System.Net.WebClient()
wc.DownloadFile(
    "https://github.com/DiffSharp/DiffSharp/releases/download/v0.7.7/DiffSharp-v0.7.7-Linux64.zip", 
    "/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7-Linux64.zip")

In [3]:
open System.Diagnostics

In [4]:
let ps = 
    ProcessStartInfo(
        FileName="unzip",
        Arguments="/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7-Linux64.zip -d /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7",
        RedirectStandardOutput=true,UseShellExecute=false)

In [5]:
let p = Process.Start(ps)
p.StandardOutput.ReadToEnd()

"Archive:  /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7-Linux64.zip
  inflating: /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/DiffSharp.dll  
  inflating: /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/DiffSharp.XML  
  inflating: /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/FSharp.Core.dll  
  inflating: /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/FSharp.Quotations.Evaluator.dll  
  inflating: /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/libopenblas.so  
  inflating: /home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/README.md  
"

In [6]:
#I "/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/"
#r "DiffSharp"
#r "FSharp.Quotations.Evaluator"

In [7]:
let openblasFromNuget = @"/home/nbuser/IfSharp/bin/packages/OpenBLAS/lib/native/bin/x64/"
let diffSharpReleaseFromZip = @"/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/"

In [8]:
open System
Environment.SetEnvironmentVariable("LD_LIBRARY_PATH", Environment.GetEnvironmentVariable("LD_LIBRARY_PATH") + ":" + diffSharpReleaseFromZip)

In [9]:
Environment.GetEnvironmentVariable("LD_LIBRARY_PATH")

"/usr/lib64/MRO-3.3.0/R-3.3.0/lib/R/lib::/home/nbuser/IfSharp/bin/packages/DiffSharp-v0.7.7/"

In [10]:
open DiffSharp.AD.Float64

In [59]:
let mergesort (arr :DV)=     
    let split (arr : D list) =
        let n = arr.Length
        arr.[0..n/2-1], arr.[n/2..n-1] 

    let rec merge l r =   
        match l, r with
            | a :: ls, b :: rs ->
                if a > b
                then b :: merge l rs
                else a :: merge ls r
            | [], r -> r
            | l, [] -> l 
            
    let rec mergeSort (arr :D list) =
        if arr.Length<2 then arr else       
        let (x, y) = split arr
        merge (mergeSort x) (mergeSort y) 
        
    let lst= arr.ToArray() |> Array.toList
    mergeSort lst |> List.toArray |> toDV

In [60]:
let rnd=System.Random()
let test= Array.init 5 (fun _-> rnd.NextDouble()) |> toDV

In [61]:
mergesort test

DV [|0.01329147956; 0.1320147026; 0.7009875526; 0.8122346009; 0.9483945975|]

In [62]:
let jacobianmergesort=jacobian mergesort
jacobianmergesort test

DM [[0.0; 0.0; 0.0; 1.0; 0.0]
    [0.0; 0.0; 0.0; 0.0; 1.0]
    [0.0; 0.0; 1.0; 0.0; 0.0]
    [1.0; 0.0; 0.0; 0.0; 0.0]
    [0.0; 1.0; 0.0; 0.0; 0.0]]